# QCodes example with Mercury iPS

## Initial instantiation/connection

In [1]:
from qcodes.instrument_drivers.oxford.MercuryiPS_VISA import MercuryiPS
from time import sleep

In [2]:
# Note that the MercuryiPS_VISA is a VISA instrument using
# a socket connection. The VISA resource name therefore 
# contains the port number and the word 'SOCKET'
mips = MercuryiPS('mips', 'TCPIP0::192.168.15.106::7020::SOCKET')

Connected to: OXFORD INSTRUMENTS MERCURY IPS (serial:170150002, firmware:2.5.08.000) in 0.10s


## Basic driver idea

The driver mainly deals with **field values** in Tesla. The driver is aware of the field values in three coordinate systems, cartesian, spherical, and cylindrical. The driver thus exposes the field coordinates x, y, z, phi, theta, rho, and r. Each coordinate comes in two versions: **target** and **measured**.

The idea is that the magnetic field is always changed in two steps; first a target is set, then the magnet is asked to ramp to said target.

## Safe regions

In addition to the safety limits baked in to the physical instrument, the driver can accept a safety limit function provided by the user. The function checks - upon receiving a new field target - whether the target is inside an allowed region.

The limit function must take input arguments Bx, By, Bz (in Tesla) and return a boolean that tells us whether that field value is safe.

In [3]:
# example: the safe region is a sphere
import numpy as np


def spherical_limit(x, y, z):
    """
    Safe region is a sphere of radius 1 T
    """
    return np.sqrt(x**2 + y**2 + z**2) <= 1

# assign the limit function (this can also be done at init)
mips.set_new_field_limits(spherical_limit)

## Two different ramps

The driver can perfom the ramp in two different ways: *simultaneous* ramping or *safe* ramping.

When simultaneously ramping, all three field components are ramped at the same time.
This method is non-blocking, and it is thus possible to query the field while it is ramping. The method does, however, **not** guarantee that the field stays inside the allowed region during the ramp. If the different axes have different ramp speeds, this is a real risk.

When safely ramping, all field components that are ramped *towards* the origin are ramped before those who are ramped *away from* the origin. The ramp is thus sequential and blocking, but if the safe region is convex (and contains the origin), you are guaranteed the the field never exceeds the safe region.

## Parameter overview

In [4]:
mips.print_readable_snapshot(update=True)

mips:
	parameter     value
--------------------------------------------------------------------------------
IDN            :	{'model': 'MERCURY IPS', 'vendor': 'OXFORD INSTRUMENTS', 'se...
phi_measured   :	0 (T)
phi_target     :	-0 (T)
r_measured     :	0 (T)
r_target       :	0 (T)
rho_measured   :	0 (T)
rho_target     :	0 (T)
theta_measured :	0 (T)
theta_target   :	0 (T)
timeout        :	5 (s)
x_measured     :	0 (T)
x_target       :	0 (T)
y_measured     :	0 (T)
y_target       :	-0 (T)
z_measured     :	-0 (T)
z_target       :	-0 (T)
mips_GRPX:
	parameter         value
--------------------------------------------------------------------------------
ATOB               :	59.451 (A/T)
current            :	0.0006 (A)
current_persistent :	0.0006 (A)
current_ramp_rate  :	0.59451 (A/s)
current_target     :	0 (A)
field              :	0 (T)
field_persistent   :	0 (T)
field_ramp_rate    :	0.01 (T/s)
field_target       :	0 (T)
ramp_status        :	HOLD 
voltage            :	-0.0003 (V)
mips_GRPY:
	

## Ramp examples

### First example: invalid targets

In [5]:
mips.x_target(1)  # so far, so good
mips.y_target(0.5)  # this takes us out of the unit sphere

ValueError: ('Cannot set y target to 0.5, that would violate the field_limits. ', 'setting mips_y_target to 0.5')

In [6]:
# reset and try in a different coordinate system
mips.x_target(0)
mips.r_target(1.1)

ValueError: ('Cannot set r target to 1.1, that would violate the field_limits. ', 'setting mips_r_target to 1.1')

### Second example: simul ramps to the origin

First we ramp the field to Bx = 1, By = 0, Bz = 0, the ramp it down to zero along theta=45, phi=30

In [7]:
mips.GRPX.field_ramp_rate(0.01)
mips.GRPY.field_ramp_rate(0.01)
mips.GRPZ.field_ramp_rate(0.01)

mips.x_target(0.1)

mips.ramp(mode='simul')

# since simul mode is non-blocking, 
# we can read out during the ramp
for n in range(20):
    print(f'Ramping X to {mips.x_target()} T, now at {mips.x_measured()} T')
    sleep(1)

Ramping X to 0.1 T, now at 0.0 T
Ramping X to 0.1 T, now at 0.0 T
Ramping X to 0.1 T, now at 0.0061 T
Ramping X to 0.1 T, now at 0.0177 T
Ramping X to 0.1 T, now at 0.0297 T
Ramping X to 0.1 T, now at 0.0408 T
Ramping X to 0.1 T, now at 0.0526 T
Ramping X to 0.1 T, now at 0.0637 T
Ramping X to 0.1 T, now at 0.0637 T
Ramping X to 0.1 T, now at 0.0756 T
Ramping X to 0.1 T, now at 0.0867 T
Ramping X to 0.1 T, now at 0.0962 T
Ramping X to 0.1 T, now at 0.0987 T
Ramping X to 0.1 T, now at 0.0995 T
Ramping X to 0.1 T, now at 0.0998 T
Ramping X to 0.1 T, now at 0.0999 T
Ramping X to 0.1 T, now at 0.1 T
Ramping X to 0.1 T, now at 0.1 T
Ramping X to 0.1 T, now at 0.1 T
Ramping X to 0.1 T, now at 0.1 T


In [8]:
mips.theta_target(45)
mips.phi_target(30)
mips.r_target(0)

mips.ramp(mode='simul')

# since simul mode is non-blocking, 
# we can read out during the ramp
for n in range(20):
    print(f'Ramping r to {mips.r_target()} T, now at {mips.r_measured()} T')
    sleep(1)

Ramping r to 0 T, now at 0.1 T
Ramping r to 0 T, now at 0.1 T
Ramping r to 0 T, now at 0.0938 T
Ramping r to 0 T, now at 0.0814 T
Ramping r to 0 T, now at 0.0702 T
Ramping r to 0 T, now at 0.0583 T
Ramping r to 0 T, now at 0.0472 T
Ramping r to 0 T, now at 0.0354 T
Ramping r to 0 T, now at 0.0354 T
Ramping r to 0 T, now at 0.0235 T
Ramping r to 0 T, now at 0.0124 T
Ramping r to 0 T, now at 0.0031 T
Ramping r to 0 T, now at 0.0012 T
Ramping r to 0 T, now at 0.0004 T
Ramping r to 0 T, now at 0.0002 T
Ramping r to 0 T, now at 0.0001 T
Ramping r to 0 T, now at 0.0001 T
Ramping r to 0 T, now at 0.0 T
Ramping r to 0 T, now at 0.0 T
Ramping r to 0 T, now at 0.0 T


### Third example: safe ramp away from the origin

At the origin, we can not **measure** what theta and phi is, but the driver remembers it.

If we ramp up again and measure, we should get back our target values. We use blocking safe ramp for this in order to not measure prematurely.

In [9]:
mips.r_target(0.05)

mips.ramp(mode='safe')

print('Ramped back out again.')
print(f'Field values are: theta: {mips.theta_measured()}, phi: {mips.phi_measured()}')

Ramped back out again.
Field values are: theta: 45.04086424151503, phi: 30.046435246810503


### That's it for now! Happy sweeping.

In [10]:
# sweep back down for good measures
mips.x_target(0)
mips.y_target(0)
mips.z_target(0)

mips.ramp(mode='safe')

mips.close()